# C-SWM 

In [1]:
#Libs
import Base: iterate, length, GC
using HDF5
using Knet
using Statistics: mean,std
using Random
using LinearAlgebra
using Images
using Plots

#Datatype
atype=KnetArray{Float32}

#Includes
include("datasets.jl")
include("cswm.jl")

## Model Definition

In [2]:
#Params
input_ch = 3
hidden_dim = 512
num_objects = 5
embedding_dim = 2
action_dim = 4
sigma = 0.5
hinge = 1.0;

In [3]:
Knet.seed!(42)

MersenneTwister(UInt32[0x0000002a], Random.DSFMT.DSFMT_state(Int32[964434469, 1073036706, 1860149520, 1073503458, 1687169063, 1073083486, -399267803, 1072983952, -909620556, 1072836235  …  -293054293, 1073002412, -1300127419, 1073642642, 1917177374, -666058738, -337596527, 1830741494, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x0000000000

## Load the pretrained model

In [14]:
model = Knet.load("model_mvmnist_slow.jld2","model")

ContrastiveSWM(EncoderCNNLarge(Param{KnetArray{Float32,4}}[P(KnetArray{Float32,4}(3,3,3,32)), P(KnetArray{Float32,4}(3,3,32,32)), P(KnetArray{Float32,4}(3,3,32,32)), P(KnetArray{Float32,4}(3,3,32,5))], Param{KnetArray{Float32,4}}[P(KnetArray{Float32,4}(1,1,32,1)), P(KnetArray{Float32,4}(1,1,32,1)), P(KnetArray{Float32,4}(1,1,32,1)), P(KnetArray{Float32,4}(1,1,5,1))], Knet.sigm, NNlib.relu), EncoderMLP(Param{KnetArray{Float32,2}}[P(KnetArray{Float32,2}(512,2500)), P(KnetArray{Float32,2}(512,512)), P(KnetArray{Float32,2}(2,512))], Param{KnetArray{Float32,1}}[P(KnetArray{Float32,1}(512)), P(KnetArray{Float32,1}(512)), P(KnetArray{Float32,1}(2))], NNlib.relu), TransitionGNN(EdgeMLP(Param{KnetArray{Float32,2}}[P(KnetArray{Float32,2}(512,4)), P(KnetArray{Float32,2}(512,512)), P(KnetArray{Float32,2}(512,512))], Param{KnetArray{Float32,1}}[P(KnetArray{Float32,1}(512)), P(KnetArray{Float32,1}(512)), P(KnetArray{Float32,1}(512))], NNlib.relu), NodeMLP(Param{KnetArray{Float32,2}}[P(KnetArray{Floa

In [5]:
#Params
EVAL_DATASET_PATH = "/home/cagan/dev/datasets/moving_mnist/movingmnist_tst_slow.h5"
EVAL_BATCH_SIZE = 100
STEP_SIZE = 5

5

In [6]:
dtst = buildMovingMNISTMultiStepPathDataset(EVAL_DATASET_PATH, EVAL_BATCH_SIZE,5);

In [7]:
o,a,no = first(dtst)

(K32(50,50,3,100)[0.0⋯], [0.42271506786346436 0.5757743120193481 … 0.097249835729599 0.2686542868614197; -0.2228550761938095 0.5261808037757874 … 0.14578384160995483 0.6992504000663757; … ; 0.12189270555973053 0.7038083672523499 … 0.5104429721832275 0.25686293840408325; 0.5433889031410217 -0.6628894209861755 … -0.07514462620019913 -0.27565547823905945]

[-0.710761547088623 0.0708666443824768 … -0.11808086931705475 0.7044579386711121; -0.5492409467697144 -0.45666012167930603 … 0.42799487709999084 0.02222311496734619; … ; 0.2906855344772339 0.31492123007774353 … -0.08657251298427582 -0.2722826600074768; 0.05790194869041443 0.2797856628894806 … 0.13601891696453094 -0.8462817072868347]

[0.5648937821388245 -0.08617222309112549 … 0.03152317926287651 -0.4274105131626129; 0.35125142335891724 -0.4507250487804413 … -0.5449708104133606 0.17186452448368073; … ; 0.5404017567634583 -0.6653268337249756 … 0.424200177192688 0.061054814606904984; -0.5472274422645569 0.6597241759300232 … -0.285541325807

In [15]:
pred_states = Any[]
next_states = Any[]

num_samples = dtst.dataset_size

for  (k, (obs, action, next_obs)) in enumerate(dtst)
    
    if k % 10 == 0
        
        println("Processed ", k ," batches")
        
    end
    #Obs => (50,50,3,100)
    #Next obs => (50,50,3,100)
    pred_state = model(obs,action[:,:,1]) + model(obs)
    
    for i in 2:STEP_SIZE

        pred_state = pred_state + model.gnn(pred_state,action[:,:,i])
        
    end
    
    pred_state = Array{Float32}(pred_state)
    next_state = Array{Float32}(model(next_obs))
    
    #Pred-state => (2,5,100)
    #Next state => (2,5,100)
    #println(pred_state)
    #println(next_state)
    
    push!(pred_states, pred_state)
    push!(next_states, next_state)
    
end

#Pred state cat => [2,5,10000]
#Next state cat => [2,5,10000]
pred_states = cat(pred_states...,dims=3)
next_states = cat(next_states...,dims=3)
    
#Flatten object/feature dimensions
pred_states = mat(pred_states)  #[10,10000]
next_states = mat(next_states)  #[10,10000]

#Calculate pairwise distances
sizes_1 = (size(pred_states)...,1)
sizes_2 = (sizes_1[1], sizes_1[3], sizes_1[2])

pred_states = reshape(pred_states, sizes_1)
next_states = reshape(next_states, sizes_2)
pred_states = repeat(pred_states, outer=[1,1,1000])
next_states = repeat(next_states, outer=[1,1000,1])

pairwise_distance_matrix = sum((pred_states - next_states).^2, dims=1)[1,:,:]

#Augment pairwise distance matrix
diag_elements = diag(pairwise_distance_matrix)
pairwise_distance_matrix = hcat(diag_elements, pairwise_distance_matrix)


labels = ones(num_samples)
hits_at_1 = 0

indices = []

for i=1:1000
    
    row = pairwise_distance_matrix[i,:]
    ind = sortperm(row)
    
    push!(indices, ind)

end

indices = vcat(indices'...)

Processed 10 batches


1000×1001 Array{Int64,2}:
   1     2   810  512  842  712  147  …  435  905  138   202   40  121  280
 602   368   641  446  379  977  347     356   44   45   783   72    5  520
   1     4   839  282  443  698  978     902  104  172   703  409  852  219
 359   457   482  673  619  739  671      79  599  537   391  941  967  956
 293     1     6  799  859  710  142     821  850  905   254  604  476  659
 912     1     7  705  770  510  477  …   91  447  315   754  129  442   48
   1     8   796  858  661  594  233     788  618  622   772  761  258  595
  96     1     9  855  752  958  169     971  757  264   472  555  412  961
 236   210   327  324  667   62  974     775  659  476   342  604  488  254
   1    11   202  754  650  214  633     810   34  676   695  199  922  281
   1    12   550  426  658  594  559  …  689  472  186   967  280  894  110
 456   814   577  757  748    1   13     436  248  263   774  704  220  691
 930    57     1   14  808  244  660     942  703  439   902  

In [16]:
num_matches = sum(labels .== indices[:,1])
println("Hits @ 1: ", num_matches/num_samples)

Hits @ 1: 0.394


In [17]:
num_matches = sum(labels .== indices[:,1:5])
println("Hits @ 5: ", num_matches/num_samples)

Hits @ 5: 0.767


In [18]:
mxval, mxindx = findmax(indices .== labels,dims=2)
ranks = [ i[2] for i in mxindx ]
reciprocal_ranks = 1 ./ranks
rr_sum = sum(reciprocal_ranks)
println("MRR: ", rr_sum/num_samples)

MRR: 0.5624242845553682
